In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/FastaiNotebook_00_load_data")' FastaiNotebook_00_load_data

Installing packages:
	.package(path: "/home/jmd/workspace/fastai/course-v3/nbs/swift/FastaiNotebook_00_load_data")
		FastaiNotebook_00_load_data
With SwiftPM flags: []
Working in: /tmp/tmpaogbyggk/swift-install
/home/jmd/swift/usr/bin/swiftc: /home/jmd/anaconda3/envs/swift/lib/libtinfo.so.6: no version information available (required by /lib/x86_64-linux-gnu/libedit.so.2)
/home/jmd/swift/usr/bin/swift: /home/jmd/anaconda3/envs/swift/lib/libuuid.so.1: no version information available (required by /home/jmd/swift/usr/bin/swift)
/home/jmd/swift/usr/bin/swift: /home/jmd/anaconda3/envs/swift/lib/libtinfo.so.6: no version information available (required by /lib/x86_64-linux-gnu/libedit.so.2)
/home/jmd/swift/usr/bin/swiftc: /home/jmd/anaconda3/envs/swift/lib/libtinfo.so.6: no version information available (required by /lib/x86_64-linux-gnu/libedit.so.2)
/home/jmd/swift/usr/bin/swift: /home/jmd/anaconda3/envs/swift/lib/libuuid.so.1: no version information available (required by /home/jmd/swift

In [2]:
//export
import Path
import TensorFlow

In [3]:
import FastaiNotebook_00_load_data

In [4]:
let zeros = Tensor<Float>(zeros: [1,4,5])
let ones  = Tensor<Float>(ones: [12,4,5])
let twos  = Tensor<Float>(repeating: 2.0, shape: [2,3,4,5])
let range = Tensor<Int32>(rangeFrom: 0, to: 32, stride: 1)

In [5]:
let xTrain = Tensor<Float>(randomNormal: [5, 784])
var weights = Tensor<Float>(randomNormal: [784, 10]) / sqrt(784)
print(weights[0])

[ 0.044765238,   0.03989295,   0.04267481, -0.071885906,   0.04634168, -0.022148294,
 -0.024864515, -0.043218795, -0.028810915, 0.0063598086]


In [6]:
// a and b are the flattened array elements, aDims/bDims are the #rows/columns of the arrays.
func swiftMatmul(a: [Float], b: [Float], aDims: (Int,Int), bDims: (Int,Int)) -> [Float] {
    assert(aDims.1 == bDims.0, "matmul shape mismatch")
    
    var res = Array(repeating: Float(0.0), count: aDims.0 * bDims.1)
    for i in 0 ..< aDims.0 {
        for j in 0 ..< bDims.1 {
            for k in 0 ..< aDims.1 {
                res[i*bDims.1+j] += a[i*aDims.1+k] * b[k*bDims.1+j]
            }
        }
    }
    return res
}

In [7]:
let flatA = xTrain[0..<5].scalars
let flatB = weights.scalars
let (aDims,bDims) = ((5, 784), (784, 10))

In [8]:
var resultArray = swiftMatmul(a: flatA, b: flatB, aDims: aDims, bDims: bDims)

In [9]:
time(repeating: 100) {
    _ = swiftMatmul(a: flatA, b: flatB, aDims: aDims, bDims: bDims)
}

average: 0.08481242999999998 ms,   min: 0.084466 ms,   max: 0.092988 ms


In [10]:
// a and b are the flattened array elements, aDims/bDims are the #rows/columns of the arrays.
func swiftMatmulUnsafe(a: UnsafePointer<Float>, b: UnsafePointer<Float>, aDims: (Int,Int), bDims: (Int,Int)) -> [Float] {
    assert(aDims.1 == bDims.0, "matmul shape mismatch")
    
    var res = Array(repeating: Float(0.0), count: aDims.0 * bDims.1)
    res.withUnsafeMutableBufferPointer { res in 
        for i in 0 ..< aDims.0 {
            for j in 0 ..< bDims.1 {
                for k in 0 ..< aDims.1 {
                    res[i*bDims.1+j] += a[i*aDims.1+k] * b[k*bDims.1+j]
                }
            }
        }
    }
    return res
}

In [11]:
time(repeating: 100) {
    _ = swiftMatmulUnsafe(a: flatA, b: flatB, aDims: aDims, bDims: bDims)
}

average: 0.04751202 ms,   min: 0.047174 ms,   max: 0.057034 ms


In [12]:
import Glibc

let ptr : UnsafeMutableRawPointer = malloc(42)

print("☠️☠️ Uninitialized garbage =", ptr.load(as: UInt8.self))

free(ptr)

☠️☠️ Uninitialized garbage = 46


In [13]:
import Python
let np = Python.import("numpy")
let pickle = Python.import("pickle")
let sys = Python.import("sys")

print("🐍list =    ", pickle.dumps([1, 2, 3]))
print("🐍ndarray = ", pickle.dumps(np.array([[1, 2], [3, 4]])))


🐍list =     b'\x80\x03]q\x00(K\x01K\x02K\x03e.'
🐍ndarray =  b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\x02K\x02\x86q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i8q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89C \x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00q\rtq\x0eb.'


In [14]:
var bias = Tensor<Float>(zeros: [10])

let m1 = Tensor<Float>(randomNormal: [5, 784])
let m2 = Tensor<Float>(randomNormal: [784, 10])

In [15]:
print("m1: ", m1.shape)
print("m2: ", m2.shape)

m1:  [5, 784]
m2:  [784, 10]


In [16]:
let small = Tensor<Float>([[1, 2],
                           [3, 4]])

print("🔢2x2:\n", small)

🔢2x2:
 [[1.0, 2.0],
 [3.0, 4.0]]


In [17]:
print("⊞ matmul:\n",  matmul(small, small))
print("\n⊞ again:\n", small • small)


⊞ matmul:
 [[ 7.0, 10.0],
 [15.0, 22.0]]

⊞ again:
 [[ 7.0, 10.0],
 [15.0, 22.0]]
